In [1]:
import numpy as np

import pyopencl as cl
import pyopencl.array as cl_array
import PyNetwork

from PyNetwork.layers.BatchNorm_GPU import BatchNorm_GPU

In [2]:
input_shape = (10,)
output_nodes = 100

layer = PyNetwork.layers.Dense(output_nodes, 'relu')
layer.build(input_shape)

platform = cl.get_platforms()
devices = platform[0].get_devices()
context = cl.Context(devices)
queue = cl.CommandQueue(context)

In [3]:
devices

[<pyopencl.Device 'Intel(R) Core(TM) i7-9750H CPU @ 2.60GHz' on 'Apple' at 0xffffffff>,
 <pyopencl.Device 'Intel(R) UHD Graphics 630' on 'Apple' at 0x1024500>,
 <pyopencl.Device 'AMD Radeon Pro 5300M Compute Engine' on 'Apple' at 0x1021e00>]

In [4]:

# Sequential Model
import numpy as np
import matplotlib.pyplot as plt

# Note: TensorFlow is not needed for PyNetwork to work. It's only used to load the dataset
import tensorflow as tf

import PyNetwork
import pyopencl as cl
import pyopencl.array as cl_array
from PyNetwork.gpu.GPUNN import GPUOPERATOR

(x_train, y_train), (x_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()

shape = (28, 28)
x_train = x_train.reshape(-1, *shape) / 255
x_test = x_test.reshape(-1, *shape) / 255

labels = np.eye(10)

y_train = labels[y_train.flatten()]
y_test = labels[y_test.flatten()]

platform = cl.get_platforms()
devices = platform[0].get_devices()
context = cl.Context(devices)
queue = cl.CommandQueue(context)
gpuoperator = GPUOPERATOR(context=context, queue=queue)

2023-04-04 12:55:36.520040: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [27]:
x_train_gpu = cl_array.to_device(queue, x_train.astype(np.float32)[:10_000])
x_test_gpu = cl_array.to_device(queue, x_test.astype(np.float32))
y_train_gpu = cl_array.to_device(queue, y_train.astype(np.float32)[:10_000])
y_test_gpu = cl_array.to_device(queue, y_test.astype(np.float32))

model_gpu = PyNetwork.Sequential_GPU(context=context,queue=queue, gpuoperator=gpuoperator)

model_gpu.add( PyNetwork.layers.Input_GPU((28, 28)) )
model_gpu.add( PyNetwork.layers.Flatten_GPU(context=context, queue=queue, gpuoperator=gpuoperator) )
model_gpu.add( PyNetwork.layers.Dense_GPU(hidden_nodes=10, activation_function='softmax', l2=0.0, l1=0.0,context=context, queue=queue, gpuoperator=gpuoperator) )
#model_gpu.add( PyNetwork.layers.BatchNorm_GPU(context=context, queue=queue, gpuoperator=gpuoperator) )
#model_gpu.add( PyNetwork.layers.Dense_GPU(hidden_nodes=10, activation_function='relu', l2=0.0, l1=0.0, context=context, queue=queue, gpuoperator=gpuoperator) )

optimizer = PyNetwork.optimizers.RMSprop_GPU(gpuoperator=gpuoperator,learning_rate=np.float32(0.0005))
model_gpu.build(loss_function='cross_entropy', optimizer=optimizer, metrics='accuracy')

In [28]:
model_gpu.evaluate(x_train_gpu, y_train_gpu)

/Users/galen/opt/anaconda3/envs/HPC4M/lib/python3.10/site-packages/pyopencl/__init__.py:270: CompilerWarning: Non-empty compiler output encountered. Set the environment variable PYOPENCL_COMPILER_OUTPUT=1 to see more.
  warn("Non-empty compiler output encountered. Set the "


'cross_entropy: 2.2591 - accuracy: 0.1375'

In [29]:
model_gpu.train(x_train_gpu, y_train_gpu, epochs=2, batch_size=128, verbose=True)

Training on 10000 samples
Epoch 1/2
cross_entropy: 20.7223 - accuracy: 0.0942
Training on 10000 samples
Epoch 2/2
cross_entropy: 20.7223 - accuracy: 0.0942


In [23]:
model_gpu.evaluate(x_train_gpu, y_train_gpu)

'cross_entropy: 20.7223 - accuracy: 0.0942'

In [24]:
model_gpu.train(x_train_gpu, y_train_gpu, epochs=10, batch_size=128, verbose=True)

Training on 10000 samples
Epoch 1/10
cross_entropy: 2.5540 - accuracy: 0.0859
Training on 10000 samples
Epoch 2/10
cross_entropy: 2.4996 - accuracy: 0.1094
Training on 10000 samples
Epoch 3/10
cross_entropy: 2.6831 - accuracy: 0.0469
Training on 10000 samples
Epoch 4/10
cross_entropy: 2.6682 - accuracy: 0.0938
Training on 10000 samples
Epoch 5/10
cross_entropy: 2.5137 - accuracy: 0.1016
Training on 10000 samples
Epoch 6/10
cross_entropy: 2.6965 - accuracy: 0.0781
Training on 10000 samples


KeyboardInterrupt: 

In [ ]:
cl_array.sum((predictions - y_train_gpu) ** 2) / float(len(predictions))

cl.Array(-2.662994e+30, dtype=float32)

In [ ]:
len(predictions)

10000